In [95]:
import pandas as pd
from datetime import timedelta, date
import re
import requests
from bs4 import BeautifulSoup

In [96]:
page = requests.get('https://www.imdb.com/search/title/?title_type=feature&genres=crime&genres=Crime&explore=genres&ref_=adv_explore_rhs')
page2 = requests.get('https://www.imdb.com/search/title/?title_type=feature&genres=crime&start=51&explore=genres&ref_=adv_nxt')
print(page)
print(page2)
soup = BeautifulSoup(page.content,'html.parser')
soup2 = BeautifulSoup(page2.content,'html.parser')

<Response [200]>
<Response [200]>


In [97]:
##movie_title and year of release
movie_title = soup.find_all('h3',class_="lister-item-header")

title=[]
yearss=[]
for movie in movie_title:
    title.append(movie.text.replace("\n","").split(".")[1].split("(")[0])
    yearss.append(re.findall('[0-9]+',(movie.text.replace("\n","").split(".")[1])))

#title
#yearss



In [104]:
movie_duration = soup.find_all('span',class_="runtime")
movie_duration2 = soup2.find_all('span',class_="runtime")
durations=[]
durations2=[]

for duration, duration2 in zip(movie_duration , movie_duration2):
    durations.append(duration.text.split('m')[0].strip(' '))
    durations.append(duration2.text.split('m')[0].strip(' '))
len(durations)

90

In [99]:
movie_discrip = soup.find_all('p',class_="text-muted")
#movie_discrip
discriptions=[]
for discrip in movie_discrip:
    discriptions.append((discrip.text))
#discriptions

In [100]:
##Movie Genre
movie_genre = soup.find_all('span',class_="genre")
genres=[]
for genre in movie_genre:
    genres.append(genre.text.replace("\n","").strip('    '))
#genres

In [101]:
##Movie Rating
movie_rating = soup.find_all('div',class_="inline-block ratings-imdb-rating")
ratings=[]
for rating in movie_rating:
    ratings.append(rating.text.replace("\n",''))
#ratings = [int(k) for k in ratings]
#ratings

##Movie Votes
movie_votes = soup.find_all('p',class_="sort-num_votes-visible")
votes=[]

for vote in movie_votes:
    votes.append(vote.text.replace("\n", '').split(':')[1].replace(",", '').strip("| Gross"))
votes_int = [int(vot) for vot in votes]

In [102]:
#Movie directors and stars
movie_director = soup.find_all('p',class_='')
directors=[]
starss=[]
for director in movie_director:
    directors.append(director.text.replace("\n",'').split('|')[0].split(':')[1].split(',')[0])
    starss.append(director.text.replace("\n",'').split('Stars:')[1])
#starss
#directors

In [103]:
movie_budgets = soup.find_all('span',name="nv")
movie_budgets
budgets=[]
# for budget in movie_budgets:
#     budgets.append(budget.text.replace("\n", '').split('$')[1])

# #votes_int = [int(vot) for vot in votes]split('M')[0]
# budgets

TypeError: find_all() got multiple values for argument 'name'

In [ ]:
# #For the next 50
# page2 = requests.get('https://www.imdb.com/search/title/?title_type=feature&genres=crime&start=51&explore=genres&ref_=adv_nxt')
# print(page)
# soup = BeautifulSoup(page2.content,'html.parser')


In [ ]:
# movie_title2 = soup.find_all('h3',class_="lister-item-header")
# print(movie_title2)
# #movie_title2[2].find_all('a')

# # title2=[]
# # for movie2 in movie_title2:
# #     title2.append(movie2.text)
# # title2